# Scientific and Research Publishing on Digital Platforms
#### Data Ingest and Processing

August 10, 2019

## Importing Modules and Defining Custom Functions
For this project, I chose to use the Python requests modules to interact with Zenodo's website and BeautifulSoup and Pandas to parse and manipulate data. I used SQLAlchemy to interact with a local database where I stored intermediate results from the website mining.

I also used GeoPy to find locations and their coordinates, cities and countries associated with research institutions using the free Nominatim geocoder primarily against Open Street Map (OSM) with Google's geocoder API as a backup. I defined two custom functions to handle geocoding for these services, and use the Folium library to review the locations on a map before exporting.

In [99]:
# Import required modules
import sys
sys.path.append("../") # go to parent dir
import requests
import time
import pandas as pd
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from IPython.display import display, HTML, clear_output, FileLink
from sqlalchemy import create_engine
from datetime import datetime, date, timedelta
import folium as folium
from geopy.geocoders import Nominatim
from geopy.geocoders import GoogleV3

# Demo flag. Set to False to run through all data
demo=True

# Define useragent and API key for location lookups. Do not leave in final code
gkey = API_KEY
ua = USER_AGENT

# Hides code in HTML output
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [100]:
# Define function to expand dataframe into new rows based on multi-value columns
def explode(df,col,delimiter=','):
    df = df.copy()
    cols1 = list(df.columns)
    cols2 = cols1.copy()
    cols2.remove(col)
    df[col] = df[col].apply(lambda x: x.split(delimiter))
    df = df[col].apply(pd.Series)\
        .merge(df, right_index = True, left_index = True)\
        .drop([col], axis = 1).melt(id_vars=cols2, value_name=col).drop(['variable'], axis = 1)
    df = df[cols1]
    return df

# Define function to search OSM geocode API for location string
def osm_geocode(location, geocoder):
    # build blank results dictionary
    result = {}
    result['name'] = location
    result['rawloc'] = ''
    result['latitude'] = ''
    result['longitude'] = ''
    result['country'] = ''
    result['city'] = ''
    # Send request to geocode API
    result['rawloc'] = geocoder.geocode(location, exactly_one=True,addressdetails=True)
    try:
        # Extract relevant fields from API response into results dictionary
        result['latitude'] = result['rawloc'].latitude
        result['longitude'] = result['rawloc'].longitude
        result['country'] = result['rawloc'].raw['address']['country_code'].upper()
        result['city'] = result['rawloc'].raw['address']['city']
    except:
        pass
    # Output result dictionary from function
    return(result)

# Define function to search OSM geocode API for location string
def google_geocode(location, geocoder):
    # build blank results dictionary
    result = {}
    result['name'] = location
    result['rawloc'] = ''
    result['latitude'] = ''
    result['longitude'] = ''
    result['country'] = ''
    result['city'] = ''
    # Send request to geocode API
    result['rawloc'] = geocoder.geocode(location, exactly_one=True)
    try:
        # Extract relevant fields from API response into results dictionary
        result['latitude'] = result['rawloc'].latitude
        result['longitude'] = result['rawloc'].longitude
        expectedResult = [d for d in result['rawloc'].raw['address_components'] if d['types'] == ['country', 'political']]
        result['country'] = expectedResult[0]['short_name']
        expectedResult = [d for d in result['rawloc'].raw['address_components'] if d['types'] == ['locality', 'political']]
        result['city'] = expectedResult[0]['short_name']
    except:
        pass
    # Output result dictionary from function
    return(result)

# Define function to extract data and store in SQLite DB
def extract(records): 
    # Extract XML fields to dictionary
    cdicts = []
    for record in records:
        cdict={}
        try:cdict['identifier'] = record.identifier.text
        except:pass
        try:cdict['subject'] = '|'.join([x.text for x in record.subjects.findChildren()])
        except:pass
        try:cdict['creator'] = '|'.join([x.creatorname.text.strip() for x in record.creators.findChildren('creator')])
        except:pass
        try:cdict['affiliation'] = '|'.join([x.affiliation.text.strip() for x in record.creators.findChildren('creator')])
        except:pass
        try:cdict['datestamp'] = record.datestamp.text
        except:pass
        try:cdict['issued'] = record.date.text
        except:pass
        try:cdict['title'] = record.title.text
        except:pass
        try:cdict['description'] = record.descriptions.text[:1000].strip()
        except:pass
        try:cdict['resourcetype'] = record.resourcetype.text.strip()
        except:pass
        try:cdict['resourcetypegeneral'] = record.resourcetype['resourcetypegeneral']
        except:pass
        try:cdict['setspec'] = record.setspec.text
        except:pass
        cdicts += [cdict]
        # Convert dictionaries to dataframe
        df = pd.DataFrame([cdict]).fillna('')
        # Store found libraries in SQL database
        engine=create_engine(sqlpath, echo=False)
        try:df.to_sql("zenodo", engine, if_exists="append", index=False)
        except:pass
    return cdicts

In [101]:
# Define template and build new local SQLite database
if demo == True:
    sqlpath = 'sqlite:///zenodo_demo.db'
else:
    sqlpath = 'sqlite:///zenodo.db'

df = pd.DataFrame(columns=['identifier', 'subject', 'creator', 'affiliation', 'datestamp', 'issued',
                          'title', 'description', 'resourceType', 'resourceTypeGeneral', 'setSpec'])
engine=create_engine(sqlpath, echo=False)
df.to_sql("zenodo", engine, if_exists="replace", index=False)

## Mining Open Publication Records Through the Zenodo OAI Interface
Zenodo provides open access through an Open Archives Initiative (AOI) XML interface which allows for retrieving structured metadata about publications. I authored a Python function to query the AOI interface for all data, and implemented a technique to query records by date. Even with splitting up the queries into individual days, the AOI interface only returns 100 records at a time. To get additional records I had to add a feature to the script to pull the temporary access token for each query, which changes after each additional 100 records and expires after 2 minutes. As the function looped through each XML response, it extracted key elements for my analysis and added it to a local SQLite database. Going back through the database, I was able to retrieve metadata for 865,366 publications over the course of two years. Pulling this data took multiple days due to rate limiting.

In [102]:
# Define start date and time delta range
sdate = date(2019,8,10)
daycount = 5

if demo==True:
    daycount=1

daydelta = timedelta(days=-1)
arcount = 0 
# Loop through start date plus each day count in delta range
for daynum in range(0,daycount):
    # Build date string and Zenodo OAI search URL
    daystring = (sdate + daydelta * daynum).strftime("%Y-%m-%d")
    print(daystring)
    search_url = 'https://zenodo.org/oai2d?verb=ListRecords&metadataPrefix=oai_datacite&from=' + \
                            daystring + 'T00:00:00Z&until=' + daystring + 'T23:59:59Z'
    print(search_url)
    # Send request to page, retrieve content as XML, and parse into object tree with BeautifulSoup
    xml_data = requests.get(search_url).content
    soup = BeautifulSoup(xml_data)
    records = soup.find_all("record")
    # Use function to extract data and store in SQLite DB
    extract(records)
    # Find total record count from OAI response
    try:recsize = int(soup.resumptiontoken['completelistsize'])
    except:recsize = len(records)
    try:print(soup.resumptiontoken['completelistsize'] + ' total results in database')
    except:pass
    rcount = len(records)
    try:print(str(len(records)) + ' results retrieved. ' + str(rcount) + ' / ' 
          + str(recsize) + ' (' + str(round(rcount/recsize*100, 1)) + '%) results retrieved total')
    except:print(str(len(records)) + ' results retrieved. ' + str(rcount) + ' / ' 
          + str(recsize) + '  results retrieved total')
    # Iterate over additional resumption tokens to get rest of data from search
    for x in range(0,500000):
        time.sleep(2)
        try:
            # Get token from current page
            resumptiontoken = soup.resumptiontoken.text
            if resumptiontoken == '': # Exit loop if there are no more tokens
                break
            print(daystring + ':' + str(x) + ': ' + resumptiontoken[:5] + '...' + resumptiontoken[-5:])
            print('Retrieving at ' + datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ") + \
                  '. Token expires ' + soup.resumptiontoken['expirationdate'])
            # Send request to page, retrieve content as XML, and parse into object tree with BeautifulSoup
            xml_data = requests.get('https://zenodo.org/oai2d?verb=ListRecords&resumptionToken=' 
                                    + resumptiontoken).content
            soup = BeautifulSoup(xml_data)
            records = soup.find_all("record")
            if len(records) == 0:
                print('retrying')
                # Send request to page, retrieve content as XML, and parse into object tree with BeautifulSoup
                xml_data = requests.get('https://zenodo.org/oai2d?verb=ListRecords&resumptionToken=' 
                                        + resumptiontoken).content
                soup = BeautifulSoup(xml_data)
                records = soup.find_all("record")
            # Use function to extract data and store in SQLite DB
            extract(records)
            rcount += len(records)
            print(str(len(records)) + ' results retrieved. ' + str(rcount) + ' / ' 
              + str(recsize) + ' (' + str(round(rcount/recsize*100, 1)) + '%) results retrieved total')
        except: # Exit loop if there are no more tokens
            break
    arcount += rcount
    
print('Script complete. ' + str(arcount) + ' results retrieved total.')

2019-08-10
https://zenodo.org/oai2d?verb=ListRecords&metadataPrefix=oai_datacite&from=2019-08-10T00:00:00Z&until=2019-08-10T23:59:59Z
794 total results in database
100 results retrieved. 100 / 794 (12.6%) results retrieved total
2019-08-10:0: .eJxV...y4YWA
Retrieving at 2019-08-12T21:09:03Z. Token expires 2019-08-12T21:10:58Z
100 results retrieved. 200 / 794 (25.2%) results retrieved total
2019-08-10:1: .eJwV...nUCYs
Retrieving at 2019-08-12T21:09:09Z. Token expires 2019-08-12T21:11:03Z
100 results retrieved. 300 / 794 (37.8%) results retrieved total
2019-08-10:2: .eJwV...HgFxk
Retrieving at 2019-08-12T21:09:14Z. Token expires 2019-08-12T21:11:09Z
100 results retrieved. 400 / 794 (50.4%) results retrieved total
2019-08-10:3: .eJwV...57fzk
Retrieving at 2019-08-12T21:09:20Z. Token expires 2019-08-12T21:11:15Z
100 results retrieved. 500 / 794 (63.0%) results retrieved total
2019-08-10:4: .eJwV...LDZQw
Retrieving at 2019-08-12T21:09:25Z. Token expires 2019-08-12T21:11:20Z
100 results retr

## Analyzing and Aggregating Data From Results
As a preliminary data exploratory analysis, I used custom SQL queries to pull data into a Pandas dataframe in Jupyter and looked for outliers and trends. I noted that resourceType’s and resourceTypeGeneral mapped together and that some types of resources may inflate statistics, such as image figures, which each receive their own DOI number in many cases. Also, in looking at affiliations I noted some non-academic organizations with obviously inflated publication numbers. For the visualization piece, I will limit the final rollups to a limited number of established academic institutions. I also needed to do similar filtering and curation of publication subjects and remove any invalid date of issue, as some dates were in the future.

### Record Types
I began with an aggregation of record types. To get the aggregations I needed, I used a direct SQL group query against the local Zenodo database with month, resource type, resource type (general), and produced a by month record count.

In [103]:
# Query local database for aggregated Zenodo publication data
engine = create_engine(sqlpath, echo=False)
df = pd.read_sql_query(
    'SELECT SUBSTR(issued,1,7) AS month, resourceTypeGeneral, resourceType, COUNT(*) AS \
    records FROM (SELECT DISTINCT identifier, resourceTypeGeneral, resourceType, issued FROM zenodo \
    WHERE issued >= "2001-01" AND issued < "2019-08") \
    GROUP BY SUBSTR(issued,1,7), resourceTypeGeneral, resourceType ORDER BY month DESC;'
    , engine).fillna('') # This SQL query aggregates all publications by month and category based on unique DOI

# Format record counts as integers
df = df.astype({"records": int})

# Fill blank resourceType
df['resourceType'] = \
    df['resourceType'].where(
        df['resourceType'] !='', df['resourceTypeGeneral'])

print(str(len(df)) + ' month/category aggregated rows created.')
display(HTML(df.head().to_html()))

95 month/category aggregated rows created.


,month,resourceTypeGeneral,resourceType,records
0,2019-07,Other,Other,1
1,2019-07,Software,Software,12
2,2019-07,Text,Book,2
3,2019-07,Text,Book section,1
4,2019-07,Text,Conference paper,5


With this by month aggregation, my next step was to generate a running total for each month and each category, as well as an "all" summarization for each category, like all months' totals for a type or all types' totals for a month. Doing this in Python is better than push the summarization to the JavaScript visualization stage since it only needs to be calculated once.

In [104]:
# Find aggregated counts for each month from all resource categories
dfg = df.groupby(['month']).agg({'records': ['sum']}).reset_index()
dfg.columns = ['month','records']
dfg['resourceTypeGeneral'] = 'All'
dfg['resourceType'] = 'All'
dfg = dfg[['month','resourceTypeGeneral','resourceType','records']]
df = pd.concat([df, dfg], ignore_index=True)

# Aggregate running totals for each category
month_list = list(set(df['month'].tolist()))
types_list = df[['resourceTypeGeneral','resourceType']].drop_duplicates().to_dict('records')
df2 = pd.DataFrame(columns=['month', 'resourceTypeGeneral', 'resourceType', 'records', 'total'])
for types in types_list:
    for month in month_list:
        total = df[(df['month'] <= month) & 
                  (df['resourceTypeGeneral'] == types['resourceTypeGeneral']) & 
                  (df['resourceType'] == types['resourceType'])]['records'].sum()
        dfp2 = df[(df['month'] == month) & 
                  (df['resourceTypeGeneral'] == types['resourceTypeGeneral']) & 
                  (df['resourceType'] == types['resourceType'])].copy()
        if len(dfp2) == 0:
            dfp2 = pd.DataFrame([[month, types['resourceTypeGeneral'], types['resourceType'], 0]], 
                                columns=['month', 'resourceTypeGeneral', 'resourceType', 'records'])
        dfp2['total'] = total
        # Concatenated back to working Dataframe
        df2 = pd.concat([df2, dfp2], ignore_index=True)
df2 = df2.astype({"records": int,"total": int})

# Find aggregated counts for all months from each categories
dfg = df2.groupby(['resourceTypeGeneral','resourceType']).agg({'records': ['sum'],
                                                               'total': ['max']}).reset_index()
dfg.columns = ['resourceTypeGeneral','resourceType','records','total']
dfg['month'] = 'All'
dfg = dfg[['month','resourceTypeGeneral','resourceType','records','total']]
# Concatenated back to working Dataframe
df2 = pd.concat([df2, dfg], ignore_index=True)
df2 = df2.sort_values(by='total', ascending=False)
df2.head()

,month,resourceTypeGeneral,resourceType,records,total
765,All,All,All,356,356
754,2019-07,All,All,102,356
744,2019-06,All,All,15,254
735,2019-05,All,All,9,239
759,2019-04,All,All,24,230


Once the results were aggregated into a single DataFrame, I converted it to a CSV for final review and a JSON for upload to the web site.

In [105]:
df2.to_csv('resource_types.csv',index=False)
df2.to_json('resource_types.json',orient='records')
FileLink('resource_types.csv')
FileLink('resource_types.json')

C:\Users\epcar\resource_types.json

### Universities
The second aggregation was affiliation, with a specific filter for Universities and Colleges. To get the aggregations I needed, I used a direct SQL group query against the local Zenodo database with month, creator, affiliation, day and month. Because a record in Zenodo has multiple affiliations, I needed to pull back the events individually and split them before aggregating them by month.

In [106]:
# Query local database for all university publications
engine = create_engine(sqlpath, echo=False)
df = pd.read_sql_query(
    'SELECT DISTINCT creator, affiliation, SUBSTR(issued,1,7) as month, SUBSTR(issued,1,10) as issued FROM zenodo \
    WHERE issued >= "2001-01" AND issued < "2019-08" AND (affiliation LIKE "%Universi%" OR affiliation LIKE "%College%") \
    ORDER BY issued DESC;'
    , engine).fillna('')

print(len(df))
display(HTML(df.head().to_html()))

101


,creator,affiliation,month,issued
0,Ashraf Afifi,"Taif University, Al-Hawiya",2019-07,2019-07-31
1,Prima Wulandari|Ridho Bayu Yefterson,Universitas Negeri Padang|Universitas Negeri Padang,2019-07,2019-07-31
2,"Rahmi Ramadhani, Ermayanti Astuti, Titin Setiawati",Universitas Potensi Utama,2019-07,2019-07-29
3,"Afsheenjinan, A.|De Silva, A. C.|Yasawardene, A.D.K.S.N.|Thayaparan, S.",University of Moratuwa|University of Moratuwa|Lady Ridgeway Hospital|University of Moratuwa,2019-07,2019-07-12
4,"Jakšová, Patrícia|Ľuptáčik, Peter|Miklisová, Dana","P.J. Šafárik University, Košice, Slovakia|P.J. Šafárik University, Košice, Slovakia|Institute of Parasitology, Slovak Academy of Sciences, Košice, Slovakia",2019-07,2019-07-08


Once the data was returned from the SQL query as a dataframe, I used the custom explode function to split the results. Because there are so many affiliations, I used another function to grab a list of just the top n Universities from the dataset, in this case 100 results.

In [107]:
# Group by specified column and create sorted top n counts
n = 100
col = 'affiliation'
df2 = df[(df[col].fillna('').str.contains('Universi'))|(df[col].fillna('').str.contains('College'))]
df2a = explode(df2,col,delimiter='|')
df2a = df2a.drop_duplicates(subset=['affiliation','month','issued'], keep='first')
df2a[col] = df2a[col].str.split(',',expand=True)[0]
df3 = df2a[(df2a[col].fillna('').str.contains('Universi'))|(df2a[col].fillna('').str.contains('College'))]
dfg = df3[[col,'issued']].groupby(col).count()
dfg.columns = ['count']
dfg = dfg.sort_values(by=['count'],ascending=False)
dfg = dfg.head(n)
print(str(len(df)) + ' records in original dataframe.')
print(str(len(df3)) + ' records in expanded/filtered dataframe.')
unilist = dfg.index.tolist()
display(HTML(dfg.head().to_html()))

cols = [col] + ['month','issued']
df_selected = df3[df3[col].isin(unilist)]
dfg = df_selected[[col,'month','issued']].groupby([col,'month']).count()
dfg.columns = ['records']
dfg = dfg.sort_values(by=['month'],ascending=False)
print(str(len(df)) + ' records in original dataframe.')
print(str(len(df3)) + ' records in expanded/filtered dataframe.')
dfg = dfg.reset_index()
df = dfg[['month','affiliation','records']].copy()
display(HTML(df.head().to_html()))

101 records in original dataframe.
43 records in expanded/filtered dataframe.


,count
affiliation,
Wageningen University,3
Universitas Ekasakti,3
Qatar University,2
Universidad Nacional de Juliaca,2
University of Moratuwa,2


101 records in original dataframe.
43 records in expanded/filtered dataframe.


,month,affiliation,records
0,2019-07,Winona State University,1
1,2019-07,University Bordeaux 2 (EA 3677 et Centre Ren? Labusqui?re) Bordeaux,1
2,2019-07,Taif University,1
3,2019-07,South East European University,1
4,2019-07,Shri Jagdishprasad Jhabarmal Tibrewala University,1


Once I had the extracted, filtered and summarized by month rollup for the top universities, I used a variation on the same rolling total function to build that dataframe. This totaler also fills in missing months for Universities with zeros.

In [108]:
# Find aggregated counts for each month from all resource categories
dfg = df.groupby(['month']).agg({'records': ['sum']}).reset_index()
dfg.columns = ['month','records']
dfg['affiliation'] = 'All'
dfg = dfg[['month','affiliation','records']]
df = pd.concat([df, dfg], ignore_index=True)

# Aggregate running totals for each category
month_list = list(set(df['month'].tolist()))
types_list = df[['affiliation']].drop_duplicates().to_dict('records')
df2 = pd.DataFrame(columns=['month', 'affiliation', 'records', 'total'])
for types in types_list:
    for month in month_list:
        total = df[(df['month'] <= month) & 
                  (df['affiliation'] == types['affiliation'])]['records'].sum()
        dfp2 = df[(df['month'] == month) & 
                  (df['affiliation'] == types['affiliation'])].copy()
        if len(dfp2) == 0:
            dfp2 = pd.DataFrame([[month, types['affiliation'], 0]], 
                                columns=['month', 'affiliation', 'records'])
        dfp2['total'] = total
        # Concatenated back to working Dataframe
        df2 = pd.concat([df2, dfp2], ignore_index=True)
df2 = df2.astype({"records": int,"total": int})

# Find aggregated counts for all months from each categories
dfg = df2.groupby(['affiliation']).agg({'records': ['sum'],
                                        'total': ['max']}).reset_index()
dfg.columns = ['affiliation','records','total']
dfg['month'] = 'All'
dfg = dfg[['month','affiliation','records','total']]
# Concatenated back to working Dataframe
df2 = pd.concat([df2, dfg], ignore_index=True)
df2 = df2.sort_values(by='total', ascending=False)
df2.head()

,month,affiliation,records,total
576,All,All,43,43
561,2019-07,All,15,43
571,2019-06,All,1,28
568,2019-04,All,6,27
567,2019-03,All,2,21


Now that the running totals were complete, I needed to geocode the locations of the Universities, with the goal of creating a map visualization in the final tool. This required the customized functions for OSM and Google's geocoder APIs. The overall function loops through the unique locations and checks OSM first. If OSM doesn't return a result it checks Google, since Google is rate limited up to a certain number of requests a day (without paying).

In [109]:
# Create Open Street Map (OSM) Nominatim and Google Geocoders
geolocator = Nominatim(user_agent=ua)
geolocator2 = GoogleV3(api_key=gkey)

# Extract location field to deduplicated list
loc_names = list(set(dfg['affiliation'].to_list()))
if 'All' in loc_names:
    loc_names.remove('All')

if demo == True:
    loc_names = loc_names[:10]
    
# Loop through locations to find coordinates
locs_clean = []
result_count = 0
for loc_name in loc_names:
    result_count += 1
    time.sleep(3)
    clear_output()
    print('Checking OSM geocoder for ' + loc_name + ' (' + str(result_count) + '/' + str(len(loc_names)) + ')')
    # Search OSM API for location
    try:
        loc_clean = osm_geocode(loc_name, geolocator)
    except:
        try:
            time.sleep(10)
            loc_clean = osm_geocode(loc_name, geolocator)
        except:
            print('Checking Google geocoder for ' + loc_name)
            loc_clean = google_geocode(loc_name, geolocator2)
    # If OSM returns no coordinates, search Google geocode API
    if loc_clean['latitude'] == '':
        print('Checking Google geocoder for ' + loc_name)
        loc_clean = google_geocode(loc_name, geolocator2)
    # Append geocoder result dictionary to results list
    locs_clean += [loc_clean]
    print(loc_clean)
print('Geocoding complete')

# Convert results dictionary list to Pandas dataframe
dfmap = pd.DataFrame(locs_clean)
dfmap = dfmap[dfmap.longitude != ''].copy()
dfmap.head()

Checking OSM geocoder for Department of Computer- Al - MuthannaUniversity. (10/10)
Checking Google geocoder for Department of Computer- Al - MuthannaUniversity.
{'name': 'Department of Computer- Al - MuthannaUniversity.', 'rawloc': Location(3043 H.M. Comer 245, 7th Ave, Tuscaloosa, AL 35487, United States, (33.2154879, -87.5444806, 0.0)), 'latitude': 33.2154879, 'longitude': -87.5444806, 'country': 'US', 'city': 'Tuscaloosa'}
Geocoding complete


,name,rawloc,latitude,longitude,country,city
0,Assam Don Bosco University,"(Assam, Vallée-du-Ntem, SU, Cameroun, (2.33656...",2.336569,10.882855,CM,
1,Taif University,"(Umm Al Qura University / Girls Campus, At Tai...",21.346149,39.928786,SA,مكة
2,Universitas Ekasakti,"(Jl. Veteran No.26B, Purus, Kec. Padang Bar., ...",-0.934607,100.354880,ID,
3,Cardiff University,"(Cardiff University, Park Place, Castle, Cardi...",51.487996,-3.179697,GB,Cardiff
4,Postgraduate Student Of Brawijaya University.,"(Jl. Veteran Malang, Ketawanggede, Kec. Lowokw...",-7.952504,112.613860,ID,


Just to verify the results of the geocoding, I dropped the coordinates from the new geocoded dataframe into a Folium map. Folium uses the same Leaflet HTML/JS embedded map that we'll use later on for the actual visualization product, so this is a helpful preview for how it will look..

In [110]:
# Create folium map of geocoded locations as circles
m = folium.Map(location=[dfmap['latitude'][0], dfmap['longitude'][0]], tiles="CartoDB positron")
for index, row in dfmap.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=5,
                        tooltip='<b>Location:</b>' + row['name'] + 
                        '<br><b>City:</b>' + row['city'] + 
                        '<br><b>Country:</b>' + row['country'],
                        fill_color="#3db7e4",
                       ).add_to(m)

# Fit map to locations
m.fit_bounds([
    [dfmap.latitude.min(),dfmap.longitude.min()],
    [dfmap.latitude.max(), dfmap.longitude.max()]
]);

# Show map
m

Once the map looked good, I merged the locations back into my summarization dataframe and created a new set of JSON and CSV.

In [111]:
dfmerged = df2.set_index('affiliation').merge(dfmap.set_index('name'), how='left', left_on='affiliation', right_on='name', right_index = True, left_index = True).reset_index()
dfmerged = dfmerged[['month','affiliation','city','country','latitude','longitude','records','total']].copy().fillna('')
print(len(dfmerged))
dfmerged = dfmerged[dfmerged['country'] != ''].copy()
dfmerged.sort_values(['records'],ascending=False).head()

612


,month,affiliation,city,country,latitude,longitude,records,total
409,2019-04,Universitas Ekasakti,,ID,-0.934607,100.355,3,3
411,All,Universitas Ekasakti,,ID,-0.934607,100.355,3,3
358,All,Taif University,مكة,SA,21.3461,39.9288,1,1
68,2019-07,Department of Computer- Al - MuthannaUniversity.,Tuscaloosa,US,33.2155,-87.5445,1,1
69,All,Department of Computer- Al - MuthannaUniversity.,Tuscaloosa,US,33.2155,-87.5445,1,1


In [112]:
dfmerged.to_csv('universities.csv',index=False)
FileLink('universities.csv')
dfmerged.to_json('universities.json',orient='records')
FileLink('universities.json')

C:\Users\epcar\universities.json

### Publication Subjects
The last topic summarization I covered was publication subjects. Similar to affiliation, a record in Zenodo can have multiple subjects. Some subjects were split with semicolons in a single field on the page, and others were combined with | from my retrieval/parsing script, so I used the explode function twice with those two delimiters and then ran my summarization. Similarly I limited this to the top 100 topics as it wouldn't be feasible to create a clean visualization of more.

In [113]:
# Query local database for all university publications
engine = create_engine(sqlpath, echo=False)
df = pd.read_sql_query(
    'SELECT DISTINCT creator, subject, SUBSTR(issued,1,7) as month, SUBSTR(issued,1,10) as issued FROM zenodo \
    WHERE issued >= "2001-01" AND issued < "2019-08" AND subject <> ""\
    ORDER BY issued DESC;'
    , engine).fillna('')

print(len(df))
display(HTML(df.head().to_html()))

# Group by specified column and create sorted top n counts
n = 100
col = 'subject'
df2 = df.copy()
df2a = explode(df2,col,delimiter='; ').fillna('')
df2 = df2a[df2a['subject'] != ''].copy()
df2a = explode(df2,col,delimiter='|').fillna('')
df2a = df2a[df2a['subject'] != ''].copy()
df2a = df2a.drop_duplicates(subset=['subject','month','issued'], keep='first')
df2a[col] = df2a[col].str.lower()
df2a[col] = df2a[col].str.split(',',expand=True)[0]
df2a = df2a[-df2a['subject'].str.contains("http|circum")].copy()
df3 = df2a.copy()
dfg = df3[[col,'issued']].groupby(col).count()
dfg.columns = ['count']
dfg = dfg.sort_values(by=['count'],ascending=False)
dfg = dfg.head(n)
print(str(len(df)) + ' records in original dataframe.')
print(str(len(df2a)) + ' records in exploded dataframe.')
print(str(len(df3)) + ' records in expanded/filtered dataframe.')
unilist = dfg.index.tolist()

cols = [col] + ['month','issued']
df_selected = df3[df3[col].isin(unilist)]
dfg = df_selected[[col,'month','issued']].groupby([col,'month']).count()
dfg.columns = ['records']
dfg = dfg.sort_values(by=['month'],ascending=False)
print(str(len(df)) + ' records in original dataframe.')
print(str(len(df3)) + ' records in expanded/filtered dataframe.')
dfg = dfg.reset_index()
df = dfg[['month','subject','records']].copy()
df.head()

148


,creator,subject,month,issued
0,Ashraf Afifi,Chaos|Diffusion|Confusion|Encryptions|Henon map,2019-07,2019-07-31
1,Салгириев Али Русланович|Баранов Андрей Владимирович|Костенко Юлия Витальевна,Россия|Северный Кавказ|политические элиты|политический процесс|структура|конфликты|напряженность|протест,2019-07,2019-07-31
2,Жуков Олег Алексеевич,экспертные системы в электроэнергетике|экспертные системы|акронимический подход,2019-07,2019-07-31
3,Prima Wulandari|Ridho Bayu Yefterson,Model Cooperative Learning|Think Talk Write|Aktivitas Belajar|Pembelajaran Sejarah,2019-07,2019-07-31
4,Григорьева Елена Эдуардовна|Сентизова Надежда Руслановна,бюджетная обеспеченность|экономические зоны Якутии|асимметрия|дифференциация|методы|оценка,2019-07,2019-07-31


148 records in original dataframe.
454 records in exploded dataframe.
454 records in expanded/filtered dataframe.
148 records in original dataframe.
454 records in expanded/filtered dataframe.


,month,subject,records
0,2019-07,state formation,1
1,2019-07,proliferation index,1
2,2019-07,uttar pradesh,1
3,2019-07,pembelajaran sejarah,1
4,2019-07,reduced basis,1


This yielded another split/filtered by month summary which I then created a running total for.

In [114]:
# Find aggregated counts for each month from all resource categories
dfg = df.groupby(['month']).agg({'records': ['sum']}).reset_index()
dfg.columns = ['month','records']
dfg['subject'] = 'All'
dfg = dfg[['month','subject','records']]
df = pd.concat([df, dfg], ignore_index=True)

# Aggregate running totals for each category
month_list = list(set(df['month'].tolist()))
types_list = df[['subject']].drop_duplicates().to_dict('records')
df2 = pd.DataFrame(columns=['month', 'subject', 'records', 'total'])
for types in types_list:
    for month in month_list:
        total = df[(df['month'] <= month) & 
                  (df['subject'] == types['subject'])]['records'].sum()
        dfp2 = df[(df['month'] == month) & 
                  (df['subject'] == types['subject'])].copy()
        if len(dfp2) == 0:
            dfp2 = pd.DataFrame([[month, types['subject'], 0]], 
                                columns=['month', 'subject', 'records'])
        dfp2['total'] = total
        # Concatenated back to working Dataframe
        df2 = pd.concat([df2, dfp2], ignore_index=True)
df2 = df2.astype({"records": int,"total": int})

# Find aggregated counts for all months from each categories
dfg = df2.groupby(['subject']).agg({'records': ['sum'],
                                        'total': ['max']}).reset_index()
dfg.columns = ['subject','records','total']
dfg['month'] = 'All'
dfg = dfg[['month','subject','records','total']]
# Concatenated back to working Dataframe
df2 = pd.concat([df2, dfg], ignore_index=True)
df2 = df2.sort_values(by='total', ascending=False)
df2.head()

,month,subject,records,total
2122,All,All,129,129
2115,2019-07,All,39,129
2110,2019-06,All,8,90
2105,2019-05,All,10,82
2119,2019-04,All,14,72


In [115]:
df2.to_csv('subjects.csv',index=False)
df2.to_json('subjects.json',orient='records')
FileLink('subjects.csv')
FileLink('subjects.json')

C:\Users\epcar\subjects.json

There were over 800,000 records on Zenodo in total for the time frame I looked at, including older publications uploaded to Zenodo, going back farther than 2000. However, for the final visualization, I limited the results to certain years with enough data to visualize, depending on the category.